In [90]:
# ===========================================> Carga librerías <====================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score, confusion_matrix, auc, plot_roc_curve, roc_curve, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, auc, RocCurveDisplay , roc_curve, classification_report

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [91]:
# ===========================================> Carga Datos <========================================================

datos = pd.read_csv('../data/train.csv', low_memory=False)

In [92]:
# ===========================================> Eliminar valores nulos <=============================================

drop_cols_min_nulls = 0.7
drop_cols_nulls = []

for col in datos.columns:
    if datos[col].isna().sum() / len(datos) >= drop_cols_min_nulls:
        drop_cols_nulls.append(col)

datos.drop(columns=drop_cols_nulls, inplace=True)

In [93]:
# ===========================================> Separar datos por tipos categoricas <================================

cat_cols = datos.select_dtypes(include=['object', 'category']).columns.to_list()
cat_cols.remove('MachineIdentifier')

datos[cat_cols] = datos[cat_cols].astype("category")

In [94]:
# ===========================================> Separar datos por tipos numericas <==================================

numeric_cols = datos.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.to_list()
numeric_cols.remove('HasDetections')

datos[numeric_cols] = datos[numeric_cols].astype("float64")

In [95]:
# ===========================================> Separar datos numericas a categoricas <==============================

numeric_cols_cat_max = 1
numeric_cols_cat = []

for col in numeric_cols:
    if (datos[col].nunique() / datos[col].count()) * 100 <= numeric_cols_cat_max:
        numeric_cols_cat.append(col) 

for col in numeric_cols_cat:
    cat_cols.append(col)
    numeric_cols.remove(col)

datos[cat_cols] = datos[cat_cols].astype("category")

In [96]:
# ===========================================> Guardamos columnas categoricas <=====================================

pickle.dump(cat_cols, open("../eda/cat_cols_raw.pkl", 'wb'))

In [97]:
# ===========================================> Guardamos columnas numericas <=======================================

pickle.dump(numeric_cols, open("../eda/numeric_cols_raw.pkl", 'wb'))

In [98]:
# ===========================================> Imputar nulos para tipos categoricas <===============================

imp_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp_cat = imp_cat.fit(datos[cat_cols])
datos[cat_cols] = imp_cat.transform(datos[cat_cols])

In [99]:
# ===========================================> Guardamos SimpleImputer para tipos categoricas <=====================

pickle.dump(imp_cat, open("../eda/symple_imputer_cat.pkl", 'wb'))

In [100]:
# ===========================================> Procesar mascaras y versiones 3 partes <=============================

mask_cols_3 = []

for col in cat_cols:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 2).all():
        mask_cols_3.append(col)

for col in mask_cols_3:
    datos[[col + "_1", col + "_2", col + "_3"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_3:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")

datos.drop(columns=mask_cols_3, inplace=True)

In [101]:
# ===========================================> Procesar mascaras y versiones 4 partes <=============================

mask_cols_4 = []

for col in cat_cols:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 3).all():
        mask_cols_4.append(col)

for col in mask_cols_4:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_4:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")

datos.drop(columns=mask_cols_4, inplace=True)

In [102]:
# ===========================================> Procesar mascaras y versiones 5 partes <=============================

mask_cols_5 = []

for col in cat_cols:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 4).all():
        mask_cols_5.append(col)

for col in mask_cols_5:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_5:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")

datos.drop(columns=mask_cols_5, inplace=True)

In [103]:
# ===========================================> Procesar mascaras y versiones 6 partes <=============================

mask_cols_6 = []

for col in cat_cols:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 5).all():
        mask_cols_6.append(col)

for c in mask_cols_6:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5", col + "_6"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_6:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")
    cat_cols.append(col + "_6")

datos.drop(columns=mask_cols_6, inplace=True)

In [104]:
# ===========================================> Procesar target encoder <============================================

cat_cols_target_encoder_min = 5
cat_cols_target_encoder = []

target_encoder = ce.TargetEncoder(handle_unknown='ignore')

for col in cat_cols:
    if datos[col].nunique() > cat_cols_target_encoder_min:
        cat_cols_target_encoder.append(col)

for col in cat_cols_target_encoder:
    cat_cols.remove(col)

target_encoder = target_encoder.fit(datos[cat_cols_target_encoder], datos['HasDetections'])
encoded_data_cat_target_encoder = target_encoder.transform(datos[cat_cols_target_encoder])

datos = pd.concat([datos.drop(columns=cat_cols_target_encoder), encoded_data_cat_target_encoder], axis=1)

In [105]:
# ===========================================> Guardamos target encoder <===========================================

pickle.dump(imp_cat, open("../eda/target_encoder_cat.pkl", 'wb'))

In [106]:
# ===========================================> Procesar onehot encoder <============================================

cat_cols_onehot_encoder_max = 5
cat_cols_onehot_encoder = []

onehot_encoder = OneHotEncoder(handle_unknown='ignore')

for col in cat_cols:
    if datos[col].nunique() <= cat_cols_onehot_encoder_max:
        cat_cols_onehot_encoder.append(col)

for col in cat_cols_onehot_encoder:
    cat_cols.remove(col)

onehot_encoder = onehot_encoder.fit(datos[cat_cols_onehot_encoder])
encoded_cols_onehot_encoder = onehot_encoder.transform(datos[cat_cols_onehot_encoder]).toarray()
encoded_data_cat_cols_onehot_encoder = pd.DataFrame(encoded_cols_onehot_encoder, columns=onehot_encoder.get_feature_names_out(cat_cols_onehot_encoder))

datos = pd.concat([datos.drop(columns=cat_cols_onehot_encoder), encoded_data_cat_cols_onehot_encoder], axis=1)

In [107]:
# ===========================================> Reiniciar Separar datos por tipos categoricas <======================

cat_cols = datos.select_dtypes(include=['object', 'category']).columns.to_list()
cat_cols.remove('MachineIdentifier')

datos[cat_cols] = datos[cat_cols].astype("category")

In [108]:
# ===========================================> Reiniciar Separar datos por tipos numericas <========================

numeric_cols = datos.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.to_list()
numeric_cols.remove('HasDetections')

datos[numeric_cols] = datos[numeric_cols].astype("float64")

In [109]:
# ===========================================> Reiniciar datos numericas a categoricas <============================

numeric_cols_cat_max = 1
numeric_cols_cat = []

for col in numeric_cols:
    if (datos[col].nunique() / datos[col].count()) * 100 <= numeric_cols_cat_max:
        numeric_cols_cat.append(col) 

for col in numeric_cols_cat:
    cat_cols.append(col)
    numeric_cols.remove(col)

datos[cat_cols] = datos[cat_cols].astype("category")

In [110]:
# ===========================================> Calculamos el rango intercuartílico <================================

intercuartilico = {}

intercuartilico["Q1"] = datos[numeric_cols].quantile(0.25)
intercuartilico["Q3"] = datos[numeric_cols].quantile(0.75)
intercuartilico["IQR"] = intercuartilico["Q3"] - intercuartilico["Q1"]

In [111]:
# ===========================================> Guardamos Intercuartilico <==========================================

pickle.dump(intercuartilico, open("../eda/intercuartilico.pkl", 'wb'))

In [112]:
# ===========================================> Tratamiento de valores atípicos <====================================

datos[numeric_cols] = datos[numeric_cols][~((datos[numeric_cols] < (intercuartilico["Q1"] - 1.5 * intercuartilico["IQR"])) |(datos[numeric_cols] > (intercuartilico["Q3"] + 1.5 * intercuartilico["IQR"]))).any(axis=1)]

In [113]:
# ===========================================> Eliminar por Matriz de correlación <=================================

drop_cols_corr = []

corr_matrix = datos[numeric_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

drop_cols_corr = [column for column in upper.columns if any(upper[column] > 0.95)]

datos.drop(columns=drop_cols_corr, inplace=True)

for col in drop_cols_corr:
    numeric_cols.remove(col)

In [114]:
# ===========================================> Imputar nulos para tipos numericas <=================================

imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')

imp_num = imp_num.fit(datos[numeric_cols])
datos[numeric_cols] = imp_num.transform(datos[numeric_cols])

In [115]:
# ===========================================> Guardamos SimpleImputer para tipos numericos <=======================

pickle.dump(imp_num, open("../eda/symple_imputer_num.pkl", 'wb'))

In [116]:
# ===========================================> Reiniciar Separar datos por tipos categoricas <======================

cat_cols = datos.select_dtypes(include=['object', 'category']).columns.to_list()
cat_cols.remove('MachineIdentifier')

datos[cat_cols] = datos[cat_cols].astype("category")

In [117]:
# ===========================================> Reiniciar Separar datos por tipos numericas <========================

numeric_cols = datos.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.to_list()
numeric_cols.remove('HasDetections')

datos[numeric_cols] = datos[numeric_cols].astype("float64")

In [118]:
# ===========================================> Reiniciar datos numericas a categoricas <============================

numeric_cols_cat_max = 1
numeric_cols_cat = []

for col in numeric_cols:
    if (datos[col].nunique() / datos[col].count()) * 100 <= numeric_cols_cat_max:
        numeric_cols_cat.append(col) 

for col in numeric_cols_cat:
    cat_cols.append(col)
    numeric_cols.remove(col)

datos[cat_cols] = datos[cat_cols].astype("category")

In [119]:
# ===========================================> Guardamos columnas categoricas <=====================================

pickle.dump(cat_cols, open("../eda/cat_cols_pre.pkl", 'wb'))

In [120]:
# ===========================================> Guardamos columnas numericas <=======================================

pickle.dump(numeric_cols, open("../eda/numeric_cols_pre.pkl", 'wb'))

In [121]:
# ===========================================> Ordenar variables <==================================================

ordered_columns = cat_cols+numeric_cols
datos = datos[["MachineIdentifier"]+ordered_columns+['HasDetections']]

In [122]:
# ===========================================> Guardamos los datos preprocesados <==================================

datos.to_csv('../data/train_pre.csv', index=False)